<a href="https://colab.research.google.com/github/Anush-Amin/Data-Analysis/blob/main/PySpark_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 12.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=d64ace2e366b125f57932ac1fa3d80cd3621007fa2ceb53662346a916b6cd5c7
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('Dataframe').getOrCreate()

In [5]:
# read the dataset
df_pyspark = spark.read.option('header','true').csv('test1.csv')

In [6]:
# Check the schema similar to info() in pandas
df_pyspark.printSchema()

root
 |-- Name : string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)



In [7]:
# In the above age and Experience is taken as string by default
## To change datatype change code to:
df_pyspark = spark.read.option('header','true').csv('test1.csv', inferSchema=True)

In [8]:
df_pyspark.printSchema()

root
 |-- Name : string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [9]:
# Another way of reading dataset
df_pyspark = spark.read.csv('test1.csv', header=True, inferSchema=True)

In [10]:
df_pyspark.printSchema()

root
 |-- Name : string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [11]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [12]:
df_pyspark.columns

['Name ', 'age', 'Experience']

In [13]:
df_pyspark.head(3)

[Row(Name ='Krish', age=31, Experience=10),
 Row(Name ='Sudhansu', age=30, Experience=8),
 Row(Name ='Sunny', age=29, Experience=4)]

In [14]:
# To pickup just name column
df_pyspark.select('Name ') #There is extra space in end of Name since its given so in csv

DataFrame[Name : string]

In [15]:
# To pickup all columns and show
df_pyspark.select(['Name ', 'age', 'Experience']).show()

+--------+---+----------+
|   Name |age|Experience|
+--------+---+----------+
|   Krish| 31|        10|
|Sudhansu| 30|         8|
|   Sunny| 29|         4|
+--------+---+----------+



In [16]:
df_pyspark.dtypes

[('Name ', 'string'), ('age', 'int'), ('Experience', 'int')]

In [17]:
df_pyspark.describe().show()

+-------+-----+----+-----------------+
|summary|Name | age|       Experience|
+-------+-----+----+-----------------+
|  count|    3|   3|                3|
|   mean| null|30.0|7.333333333333333|
| stddev| null| 1.0|3.055050463303893|
|    min|Krish|  29|                4|
|    max|Sunny|  31|               10|
+-------+-----+----+-----------------+



In [18]:
# Add columns in data frame
## this is not inplace operation so we need to assign it
df_pyspark = df_pyspark.withColumn('Experience after 2 years', df_pyspark['Experience']+2)

In [19]:
df_pyspark.show()

+--------+---+----------+------------------------+
|   Name |age|Experience|Experience after 2 years|
+--------+---+----------+------------------------+
|   Krish| 31|        10|                      12|
|Sudhansu| 30|         8|                      10|
|   Sunny| 29|         4|                       6|
+--------+---+----------+------------------------+



In [20]:
# Drop the columns
df_pyspark = df_pyspark.drop('Experience after 2 years')

In [21]:
# Rename the column
## Not a inplace operation
df_pyspark.withColumnRenamed('Name ', 'New Name').show()

+--------+---+----------+
|New Name|age|Experience|
+--------+---+----------+
|   Krish| 31|        10|
|Sudhansu| 30|         8|
|   Sunny| 29|         4|
+--------+---+----------+



**Handling Missing value**

In [23]:
df_pyspark2 = spark.read.csv('test2.csv', header=True, inferSchema=True)

In [24]:
df_pyspark2.show()

+--------+----+----------+------+
|   Name | age|Experience|Salary|
+--------+----+----------+------+
|   Krish|  31|        10| 30000|
|Sudhansu|  30|         8| 25000|
|   Sunny|  29|         4| 20000|
|    Paul|  24|         3| 20000|
|  Harsha|  21|         1| 15000|
| Shubham|  23|         2| 18000|
|  Mahesh|null|      null| 40000|
|    null|  34|        10| 38000|
|    null|  36|      null|  null|
+--------+----+----------+------+



In [25]:
df_pyspark2.printSchema()

root
 |-- Name : string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [26]:
# Drops all rows that has null
## Not inplace opperation
df_pyspark2.na.drop().show()

+--------+---+----------+------+
|   Name |age|Experience|Salary|
+--------+---+----------+------+
|   Krish| 31|        10| 30000|
|Sudhansu| 30|         8| 25000|
|   Sunny| 29|         4| 20000|
|    Paul| 24|         3| 20000|
|  Harsha| 21|         1| 15000|
| Shubham| 23|         2| 18000|
+--------+---+----------+------+



In [27]:
# how == any
## this will drop rows which has null value in any columns
## Not inplace opperation
df_pyspark2.na.drop(how='any').show()

+--------+---+----------+------+
|   Name |age|Experience|Salary|
+--------+---+----------+------+
|   Krish| 31|        10| 30000|
|Sudhansu| 30|         8| 25000|
|   Sunny| 29|         4| 20000|
|    Paul| 24|         3| 20000|
|  Harsha| 21|         1| 15000|
| Shubham| 23|         2| 18000|
+--------+---+----------+------+



In [28]:
# how == any with thresh==2
## this will drop rows which has null value in greater than 2 columns
## Not inplace opperation
df_pyspark2.na.drop(how='any', thresh=2).show()

+--------+----+----------+------+
|   Name | age|Experience|Salary|
+--------+----+----------+------+
|   Krish|  31|        10| 30000|
|Sudhansu|  30|         8| 25000|
|   Sunny|  29|         4| 20000|
|    Paul|  24|         3| 20000|
|  Harsha|  21|         1| 15000|
| Shubham|  23|         2| 18000|
|  Mahesh|null|      null| 40000|
|    null|  34|        10| 38000|
+--------+----+----------+------+



In [29]:
# Subset
## this will drop rows which has null value in Experience column
## Not inplace opperation
df_pyspark2.na.drop(how='any', subset=['Experience']).show()

+--------+---+----------+------+
|   Name |age|Experience|Salary|
+--------+---+----------+------+
|   Krish| 31|        10| 30000|
|Sudhansu| 30|         8| 25000|
|   Sunny| 29|         4| 20000|
|    Paul| 24|         3| 20000|
|  Harsha| 21|         1| 15000|
| Shubham| 23|         2| 18000|
|    null| 34|        10| 38000|
+--------+---+----------+------+



In [30]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols = ['age', 'Experience', 'Salary'],
    outputCols = ['{}_imputed'.format(c) for c in ['age', 'Experience', 'Salary']]
).setStrategy('mean')

In [31]:
# Add imputation cols to df
imputer.fit(df_pyspark2).transform(df_pyspark2).show()

+--------+----+----------+------+-----------+------------------+--------------+
|   Name | age|Experience|Salary|age_imputed|Experience_imputed|Salary_imputed|
+--------+----+----------+------+-----------+------------------+--------------+
|   Krish|  31|        10| 30000|         31|                10|         30000|
|Sudhansu|  30|         8| 25000|         30|                 8|         25000|
|   Sunny|  29|         4| 20000|         29|                 4|         20000|
|    Paul|  24|         3| 20000|         24|                 3|         20000|
|  Harsha|  21|         1| 15000|         21|                 1|         15000|
| Shubham|  23|         2| 18000|         23|                 2|         18000|
|  Mahesh|null|      null| 40000|         28|                 5|         40000|
|    null|  34|        10| 38000|         34|                10|         38000|
|    null|  36|      null|  null|         36|                 5|         25750|
+--------+----+----------+------+-------

**PySpark operations**

In [32]:
df_pyspark3 = spark.read.csv('test3.csv', header=True, inferSchema=True)

In [33]:
df_pyspark3.show()

+--------+---+----------+------+
|    Name|age|Experience|Salary|
+--------+---+----------+------+
|   Krish| 31|        10| 30000|
|Sudhansu| 30|         8| 25000|
|   Sunny| 29|         4| 20000|
|    Paul| 24|         3| 20000|
|  Harsha| 21|         1| 15000|
| Shubham| 23|         2| 18000|
+--------+---+----------+------+



In [34]:
# Salary of the people less than equal to 20000
df_pyspark3.filter('Salary<=20000').show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



In [35]:
# Above code can also be written as
df_pyspark3.filter(df_pyspark3['Salary']<=20000).show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



In [36]:
df_pyspark3.filter((df_pyspark3['Salary']<=20000) & 
                   (df_pyspark3['Salary']>=15000)).show()
## both conditions should be written within bracket orelse will throw error

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



In [37]:
# Not opertion also known as inverse filter
## ~ is the symbol
df_pyspark3.filter(~(df_pyspark3['Salary']<=20000)).show()

+--------+---+----------+------+
|    Name|age|Experience|Salary|
+--------+---+----------+------+
|   Krish| 31|        10| 30000|
|Sudhansu| 30|         8| 25000|
+--------+---+----------+------+



**GroupBy and Aggregate functions**

In [38]:
df_pyspark4 = spark.read.csv('test4.csv', header=True, inferSchema=True)

In [54]:
df_pyspark4.show()

+---------+------------+------+
|     Name| Departments|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



In [39]:
df_pyspark4.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- salary: integer (nullable = true)



In [45]:
# Groupby
## Grouped to find maximum salary
df_pyspark4.groupby('Name').sum().show()

+---------+-----------+
|     Name|sum(salary)|
+---------+-----------+
|Sudhanshu|      35000|
|    Sunny|      12000|
|    Krish|      19000|
|   Mahesh|       7000|
+---------+-----------+



In [46]:
# Groupby departments which gives maximum salary
df_pyspark4.groupby('Departments').sum().show()

+------------+-----------+
| Departments|sum(salary)|
+------------+-----------+
|         IOT|      15000|
|    Big Data|      15000|
|Data Science|      43000|
+------------+-----------+



In [47]:
# Groupby departments to show mean salary
df_pyspark4.groupby('Departments').mean().show()

+------------+-----------+
| Departments|avg(salary)|
+------------+-----------+
|         IOT|     7500.0|
|    Big Data|     3750.0|
|Data Science|    10750.0|
+------------+-----------+



In [48]:
df_pyspark4.groupby('Departments').count().show()

+------------+-----+
| Departments|count|
+------------+-----+
|         IOT|    2|
|    Big Data|    4|
|Data Science|    4|
+------------+-----+



In [53]:
#find total expenditure
df_pyspark4.agg({'Salary': 'sum'}).show()

+-----------+
|sum(Salary)|
+-----------+
|      73000|
+-----------+

